# DAX Benchmarks

In [14]:
import json
import numpy as np
import pandas as pd
import sempy.fabric as fabric
import statistics as stats
import time
import uuid

from pyspark.sql import functions as F

StatementMeta(, 6dad8994-4bfa-4ed3-9d38-515b0538de6d, 16, Finished, Available, Finished)

## Parameters

In [15]:
WORKSPACE = None  # same workspace as notebook/lakehouse; set name/ID if needed

DATASETS = {
    "SS": "Star Schema",
    "USS": "Unified Star Schema",
}

MIN_MEASURED = 5        # don't stop before this many measured runs
MAX_MEASURED = 100      # hard cap
WARMUP = 2              # still run warmups, but store them
CV_TARGET = 0.05        # 5% coefficient of variation
ERROR_LIMIT = 3         # if a query errors too much, stop early

MAX_RESULT_ROWS_TO_STORE = 200_000  # safety cap

OUT_DB = "benchmarks"
RUNS_TABLE = "runs"
RESULTS_TABLE = "results"

StatementMeta(, 6dad8994-4bfa-4ed3-9d38-515b0538de6d, 17, Finished, Available, Finished)

## Query Suite

In [16]:
QUERY_TEMPLATES = [
    ("P1_store_sales_by_year", """
EVALUATE
SUMMARIZECOLUMNS(
    'dim__date'[year],
    "Store Sales", SUM('{fact}'[_measure__store_sales__ext_sales_price])
)
ORDER BY 'dim__date'[year]
"""),

    ("P2_store_sales_by_year_category", """
EVALUATE
SUMMARIZECOLUMNS(
    'dim__date'[year],
    'dim__item'[category],
    "Store Sales", SUM('{fact}'[_measure__store_sales__ext_sales_price])
)
ORDER BY 'dim__date'[year], 'dim__item'[category]
"""),

    ("P3_store_sales_many_groups", """
EVALUATE
SUMMARIZECOLUMNS(
    'dim__date'[year],
    'dim__item'[category],
    'dim__store'[_key__dim__store],
    "Store Sales", SUM('{fact}'[_measure__store_sales__ext_sales_price])
)
"""),

    ("P4_top100_items_store_sales_2002", """
EVALUATE
TOPN(
    100,
    SUMMARIZECOLUMNS(
        'dim__item'[_key__dim__item],
        'dim__item'[category],
        TREATAS( {{ 2002 }}, 'dim__date'[year] ),
        "Store Sales", SUM('{fact}'[_measure__store_sales__ext_sales_price])
    ),
    [Store Sales], DESC,
    'dim__item'[_key__dim__item], ASC
)
"""),

    ("P5_net_store_sales_by_month", """
EVALUATE
SUMMARIZECOLUMNS(
    'dim__date'[year],
    'dim__date'[month_of_year],
    "Sales",   SUM('{fact}'[_measure__store_sales__ext_sales_price]),
    "Returns", SUM('{fact}'[_measure__store_returns__return_amt]),
    "Net",
        SUM('{fact}'[_measure__store_sales__ext_sales_price])
        - SUM('{fact}'[_measure__store_returns__return_amt])
)
ORDER BY 'dim__date'[year], 'dim__date'[month_of_year]
"""),

    ("P7_total_sales_all_channels_by_year", """
EVALUATE
SUMMARIZECOLUMNS(
    'dim__date'[year],
    "Total Sales",
        SUM('{fact}'[_measure__store_sales__ext_sales_price])
        + SUM('{fact}'[_measure__web_sales__ext_sales_price])
        + SUM('{fact}'[_measure__catalog_sales__ext_sales_price])
)
ORDER BY 'dim__date'[year]
"""),

    ("P9_inventory_wh_category_year_2002", """
EVALUATE
SUMMARIZECOLUMNS(
    'dim__warehouse'[_key__dim__warehouse],
    'dim__item'[category],
    TREATAS( {{ 2002 }}, 'dim__date'[year] ),
    "Qty On Hand", SUM('{fact}'[_measure__inventory__qty_on_hand])
)
"""),

    # -------------------------
    # NEW: Time intelligence (YoY)
    # -------------------------
    ("P10_store_sales_yoy_by_year", """
EVALUATE
ADDCOLUMNS(
    SUMMARIZECOLUMNS(
        'dim__date'[year],
        "Sales", SUM('{fact}'[_measure__store_sales__ext_sales_price])
    ),
    "Sales PY",
        CALCULATE(
            SUM('{fact}'[_measure__store_sales__ext_sales_price]),
            SAMEPERIODLASTYEAR('dim__date'[date])
        ),
    "YoY Δ",
        SUM('{fact}'[_measure__store_sales__ext_sales_price])
        - CALCULATE(
            SUM('{fact}'[_measure__store_sales__ext_sales_price]),
            SAMEPERIODLASTYEAR('dim__date'[date])
        ),
    "YoY %",
        DIVIDE(
            SUM('{fact}'[_measure__store_sales__ext_sales_price])
            - CALCULATE(
                SUM('{fact}'[_measure__store_sales__ext_sales_price]),
                SAMEPERIODLASTYEAR('dim__date'[date])
            ),
            CALCULATE(
                SUM('{fact}'[_measure__store_sales__ext_sales_price]),
                SAMEPERIODLASTYEAR('dim__date'[date])
            )
        )
)
ORDER BY 'dim__date'[year]
"""),
]

# SS queries reference multiple physical fact tables; USS uses only _bridge.
FACT_FOR_MEASURE_PREFIX = {
    "SS": {
        "_measure__store_sales__": "fact__store_sales",
        "_measure__store_returns__": "fact__store_returns",
        "_measure__web_sales__": "fact__web_sales",
        "_measure__catalog_sales__": "fact__catalog_sales",
        "_measure__inventory__": "fact__inventory",
    },
    "USS": {
        "_measure__store_sales__": "_bridge",
        "_measure__store_returns__": "_bridge",
        "_measure__web_sales__": "_bridge",
        "_measure__catalog_sales__": "_bridge",
        "_measure__inventory__": "_bridge",
    }
}

def materialize_queries(query_set: str):
    """
    Expands QUERY_TEMPLATES into list[(name, dax)] for query_set ("SS" or "USS"),
    swapping only the fact source(s).
    """
    mapping = FACT_FOR_MEASURE_PREFIX[query_set]

    out = []
    for name, tmpl in QUERY_TEMPLATES:
        q = tmpl.format(fact="__FACT__")

        for prefix, table in mapping.items():
            q = q.replace(f"SUM('__FACT__'[{prefix}", f"SUM('{table}'[{prefix}")

        if "__FACT__" in q:
            raise ValueError(f"Unmapped measure prefix in query template {name} for set={query_set}")

        out.append((name, q))
    return out

SS_QUERIES = materialize_queries("SS")
USS_QUERIES = materialize_queries("USS")

# -------------------------
# NEW: Distinct customers with orders
# (SS: summarize per fact first, then union; USS: peripheral filter)
# -------------------------
SS_QUERIES.append((
    "P11_distinct_customers_with_orders",
    """
EVALUATE
VAR StoreCustomers =
    SUMMARIZE(
        'fact__store_sales',
        'fact__store_sales'[_key__dim__customer]
    )
VAR WebCustomers =
    SUMMARIZE(
        'fact__web_sales',
        'fact__web_sales'[_key__dim__customer]
    )
VAR CatalogCustomers =
    SUMMARIZE(
        'fact__catalog_sales',
        'fact__catalog_sales'[_key__dim__customer]
    )
VAR CustomersWithOrders =
    DISTINCT(
        UNION(
            StoreCustomers,
            WebCustomers,
            CatalogCustomers
        )
    )
RETURN
ROW(
    "Distinct Customers With Orders",
    COUNTROWS(CustomersWithOrders)
)
"""
))

USS_QUERIES.append((
    "P11_distinct_customers_with_orders",
    """
EVALUATE
VAR CustomersWithOrders =
    CALCULATETABLE(
        DISTINCT(SELECTCOLUMNS('_bridge', "CustomerKey", '_bridge'[_key__dim__customer])),
        KEEPFILTERS('_bridge'[peripheral] IN { "fact__store_sales", "fact__web_sales", "fact__catalog_sales" })
    )
RETURN
ROW(
    "Distinct Customers With Orders",
    COUNTROWS(CustomersWithOrders)
)
"""
))

QUERY_SETS = {
    "SS": SS_QUERIES,
    "USS": USS_QUERIES
}

StatementMeta(, 6dad8994-4bfa-4ed3-9d38-515b0538de6d, 18, Finished, Available, Finished)

## Helpers

In [17]:
def _stable_hash_df(df: pd.DataFrame) -> str:
    if df is None:
        return None
    df2 = df.copy().reindex(sorted(df.columns), axis=1)
    h = pd.util.hash_pandas_object(df2, index=True).values.astype("uint64")
    folded = np.bitwise_xor.reduce(h) if len(h) else np.uint64(0)
    return str(int(folded))

def _df_to_json_payload(df: pd.DataFrame, max_rows: int | None):
    if df is None:
        return None
    truncated = False
    if max_rows is not None and len(df) > max_rows:
        df = df.head(max_rows)
        truncated = True
    return {
        "truncated": truncated,
        "rows": int(len(df)),
        "cols": int(df.shape[1]),
        "columns": list(df.columns),
        "data": json.loads(df.to_json(orient="records", date_format="iso"))
    }

def run_one_dax(dataset_name: str, dax: str, workspace=None) -> pd.DataFrame:
    return fabric.evaluate_dax(dataset=dataset_name, dax_string=dax, workspace=workspace)

StatementMeta(, 6dad8994-4bfa-4ed3-9d38-515b0538de6d, 19, Finished, Available, Finished)

In [18]:
def cv(values: list[float]) -> float:
    if len(values) < 2:
        return float("inf")
    mean = stats.fmean(values)
    if mean == 0:
        return float("inf")
    return stats.pstdev(values) / mean

def run_query_adaptive(dataset_name: str, suite_key: str, qname: str, dax: str, workspace=None):
    """
    Runs warmups + measured repetitions until stable by CV_TARGET or cap reached.
    Stores all runs (warmup + measured).
    Returns: (runs_rows, results_rows, stop_reason)
    """
    runs_rows_local = []
    results_rows_local = []

    measured_durations = []
    errors = 0
    seq = 0
    stop_reason = None

    def record(run_phase: str, rep_n: int, duration_ms: float, df, err: str | None):
        nonlocal seq
        seq += 1
        row_count = None if df is None else int(len(df))
        col_count = None if df is None else int(df.shape[1])
        result_hash = None if df is None else _stable_hash_df(df)
        payload = _df_to_json_payload(df, MAX_RESULT_ROWS_TO_STORE)

        runs_rows_local.append({
            "run_id": run_id,
            "started_utc": started_utc,
            "dataset": dataset_name,
            "suite": suite_key,
            "query_name": qname,
            "phase": run_phase,          # warmup / measured
            "repeat_n": rep_n,           # within phase
            "seq_in_query": seq,         # across phases
            "duration_ms": float(duration_ms),
            "row_count": row_count,
            "col_count": col_count,
            "result_hash": result_hash,
            "error": err,
        })

        results_rows_local.append({
            "run_id": run_id,
            "dataset": dataset_name,
            "suite": suite_key,
            "query_name": qname,
            "phase": run_phase,
            "repeat_n": rep_n,
            "seq_in_query": seq,
            "payload_json": None if payload is None else json.dumps(payload)
        })

    # ---- Warmups (stored) ----
    for w in range(1, WARMUP + 1):
        t0 = time.perf_counter()
        df = None
        err = None
        try:
            df = run_one_dax(dataset_name, dax, workspace=workspace)
        except Exception as e:
            err = str(e); errors += 1
        t1 = time.perf_counter()
        record("warmup", w, (t1 - t0) * 1000.0, df, err)

    # ---- Measured runs (adaptive) ----
    m = 0
    while True:
        m += 1
        t0 = time.perf_counter()
        df = None
        err = None
        try:
            df = run_one_dax(dataset_name, dax, workspace=workspace)
        except Exception as e:
            err = str(e); errors += 1
        t1 = time.perf_counter()

        dur_ms = (t1 - t0) * 1000.0
        record("measured", m, dur_ms, df, err)

        if err is None:
            measured_durations.append(dur_ms)

        # stop conditions
        if errors >= ERROR_LIMIT:
            stop_reason = "error_limit"
            break

        if m >= MAX_MEASURED:
            stop_reason = "max_measured"
            break

        if len(measured_durations) >= MIN_MEASURED:
            current_cv = cv(measured_durations)
            if current_cv <= CV_TARGET:
                stop_reason = f"cv<={CV_TARGET}"
                break

    # annotate stop reason on the LAST run row for this query
    runs_rows_local[-1]["stop_reason"] = stop_reason

    return runs_rows_local, results_rows_local, stop_reason

StatementMeta(, 6dad8994-4bfa-4ed3-9d38-515b0538de6d, 20, Finished, Available, Finished)

## Execute

In [19]:
run_id = str(uuid.uuid4())
started_utc = pd.Timestamp.utcnow().isoformat()

runs_rows = []
results_rows = []

for suite_key, dataset_name in DATASETS.items():
    for (qname, dax) in QUERY_SETS[suite_key]:
        rr, resr, stop_reason = run_query_adaptive(
            dataset_name=dataset_name,
            suite_key=suite_key,
            qname=qname,
            dax=dax,
            workspace=WORKSPACE
        )
        runs_rows.extend(rr)
        results_rows.extend(resr)
        print(f"[{suite_key}] {qname} stop_reason={stop_reason}")

StatementMeta(, 6dad8994-4bfa-4ed3-9d38-515b0538de6d, 21, Finished, Available, Finished)

[SS] P1_store_sales_by_year stop_reason=cv<=0.05
[SS] P2_store_sales_by_year_category stop_reason=max_measured
[SS] P3_store_sales_many_groups stop_reason=max_measured
[SS] P4_top100_items_store_sales_2002 stop_reason=max_measured
[SS] P5_net_store_sales_by_month stop_reason=max_measured
[SS] P7_total_sales_all_channels_by_year stop_reason=max_measured
[SS] P9_inventory_wh_category_year_2002 stop_reason=max_measured
[SS] P10_store_sales_yoy_by_year stop_reason=max_measured
[SS] P11_distinct_customers_with_orders stop_reason=cv<=0.05
[USS] P1_store_sales_by_year stop_reason=max_measured
[USS] P2_store_sales_by_year_category stop_reason=cv<=0.05
[USS] P3_store_sales_many_groups stop_reason=max_measured
[USS] P4_top100_items_store_sales_2002 stop_reason=max_measured
[USS] P5_net_store_sales_by_month stop_reason=max_measured
[USS] P7_total_sales_all_channels_by_year stop_reason=max_measured
[USS] P9_inventory_wh_category_year_2002 stop_reason=max_measured
[USS] P10_store_sales_yoy_by_year 

## Summarize

In [35]:
runs_df = pd.DataFrame(runs_rows)

StatementMeta(, 6dad8994-4bfa-4ed3-9d38-515b0538de6d, 37, Finished, Available, Finished)

In [56]:
summary = (
    runs_df
    .query("phase == 'measured' and error.isna()")
    .groupby(["dataset", "query_name"], as_index=False)
    .agg(
        avg_duration_ms=("duration_ms", "mean"),
        median_duration_ms=("duration_ms", "median"),
        runs=("duration_ms", "count")
    )
)

display(summary)

StatementMeta(, 6dad8994-4bfa-4ed3-9d38-515b0538de6d, 58, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bbc573ef-baa8-4d57-bc0b-086b68c8f4ae)

In [51]:
(
    runs_df
    .query("phase == 'measured' and error.isna()")
    .groupby(["suite", "query_name"], as_index=False)
    .agg(median_duration_ms=("duration_ms", "median"))
    .pivot(index="query_name", columns="suite", values="median_duration_ms")
).bar()

StatementMeta(, 6dad8994-4bfa-4ed3-9d38-515b0538de6d, 53, Finished, Available, Finished)

AttributeError: 'DataFrame' object has no attribute 'bar'